In [ ]:
# 02. Enriquecimento de Dados: chuva

1. Bibliotecas e DataFrame

In [1]:
import requests
import pandas as pd
import os

df = pd.read_parquet("../dados/intermediarios/acidentes_tratado.parquet")

2. Coordenadas por região

In [ ]:
coord = {
    "NORTE":  (-29.987, -51.165),
    "LESTE":  (-30.040, -51.160),
    "CENTRO": (-30.027, -51.220),
    "SUL":    (-30.120, -51.230)
}

anos = [2020, 2021, 2022, 2023, 2024]
dfs_all = []

os.makedirs("..dados/intermediarios/clima", exist_ok=True)

3. Cria função para API (por região, por ano para não quebrar)

In [4]:
# Consulta API Open Meteo
def chuva_regiao_ano(regiao, lat, lon, ano):
    data_inicio = f"{ano}-01-01"
    data_fim = f"{ano}-12-31"
    
    url = (
        "https://archive-api.open-meteo.com/v1/archive"
        f"?latitude={lat}&longitude={lon}"
        f"&start_date={data_inicio}&end_date={data_fim}"
        "&hourly=precipitation"
        "&timezone=America/Sao_Paulo"
    )
    
    resposta = requests.get(url).json()
    
    if "hourly" in resposta and "time" in resposta["hourly"]:
        df_meteo = pd.DataFrame({
            "time": pd.to_datetime(resposta["hourly"]["time"], errors="coerce"),
            "precipitation": resposta["hourly"]["precipitation"],
        })
        df_meteo["regiao"] = regiao
        return df_meteo
    else:
        print(f"⚠️ Sem dados para {regiao}-{ano}")
        return pd.DataFrame(columns=["time","precipitation","regiao"])


In [5]:
df_teste = chuva_regiao_ano("CENTRO", -30.027, -51.220, 2022)
print(df_teste.head())
print(len(df_teste))


                 time  precipitation  regiao
0 2022-01-01 00:00:00            0.0  CENTRO
1 2022-01-01 01:00:00            0.0  CENTRO
2 2022-01-01 02:00:00            0.0  CENTRO
3 2022-01-01 03:00:00            0.0  CENTRO
4 2022-01-01 04:00:00            0.0  CENTRO
8760
